In [7]:
import numpy as np
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.optim.lr_scheduler import OneCycleLR
import matplotlib.pyplot as plt
from tqdm import tqdm
from small2DNet import small2DNet
from util import add_color, colorize, colorize_gaussian, calculate_correct_loss
from colorMNist import colorMNist
import random
import colorsys
import pickle
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [3]:
# MNIST DATASET

# Load data from pickle file
cmnist_train, cmnist_val, cmnist_test = pickle.load(open("custom_datasets/1k/cmnist_gaussian_12.pkl", "rb"))
print(len(cmnist_train), len(cmnist_val), len(cmnist_train) + len(cmnist_val))

# Create datasets
train_dataset = colorMNist(cmnist_train)
val_dataset = colorMNist(cmnist_val)
test_dataset = colorMNist(cmnist_test)

# Dataloaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                               shuffle=True, num_workers = 0)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
                                               shuffle=True, num_workers = 0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
                                               shuffle=False, num_workers = 0)

800 200 1000


In [ ]:
# CIFAR10 DATASET

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='datasets/cifar10', train=True, download=False, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='datasets/cifar10', train=False, download=False, transform=transform)

# Create the 80/20 train/val split
train_split = int(0.8 * len(trainset))
# Train dataset
train_dataset = torch.utils.data.Subset(trainset, [i for i in range(train_split)])
# Validation dataset
val_dataset = torch.utils.data.Subset(trainset, [i for i in range(train_split, len(trainset))])
print(len(train_dataset), len(val_dataset), len(train_dataset) + len(val_dataset))

train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [31]:
# Layers of the model
model_layers = [28, "M", 55, "M", 111,"M"]
# Set seed
torch.manual_seed(2)
# Create model
model = small2DNet(model_layers, model_layers[-2], [1, 1])
# Load file and save file
# lfile = "Gaussian2D_12"
sfile = "Gaussian2D_uniform"
# Load model
# model.load_state_dict(torch.load('model_saves/new_fair/'+ lfile + '.pth'))
# Put model on gpu
model.cuda()
# Loss function
loss_fn = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.5)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_dataloader), epochs=20)

In [ ]:
# Freeze weights
children = [x for x in model.children()]
for x in children[0]:
    for param in x.parameters():
        param.requires_grad = False

In [32]:
# Number of epochs to train
epochs = 20

# Placeholder variables to put training and validation accuracies and losses per epoch
train_accuracies = []
train_losses = []
val_accuracies = []
val_losses = []

for epoch in tqdm(range(epochs), total=epochs, desc='Training'):
    # print("Epoch", epoch + 1, "/", epochs)
    
    # Update learning rate
    # if (epoch + 1) % 20 == 0:
    #     for g in optimizer.param_groups:
    #         g['lr'] /= 10
    
    # Put model on training mode
    model.train()
    train_total_correct = 0
    train_total_loss = []
    
    for (images, labels) in train_dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Calculate number correct and loss in batch
        correct, loss = calculate_correct_loss(model, loss_fn, images, labels)
        
        # Backpropagation
        loss.backward()
        # Step function
        optimizer.step()
        
        scheduler.step()
        # print(scheduler.get_last_lr())
        
        # Update amount correct and loss with current batch
        train_total_correct += correct
        train_total_loss.append(loss.detach().item())
        
    # Append epoch accuracy and loss
    train_accuracies.append(train_total_correct / len(train_dataset))
    train_losses.append(sum(train_total_loss) / len(train_total_loss))
    
    # Put model on evaluation mode
    model.eval()
    val_total_correct = 0
    val_total_loss = []
    
    # Without gradient calculation
    with torch.no_grad():
        for (images, labels) in val_dataloader:
        
            # Calculate number correct and loss in batch
            correct, loss = calculate_correct_loss(model, loss_fn, images, labels)

            # Update amount correct and loss with current batch
            val_total_correct += correct
            val_total_loss.append(loss.detach().item())

    # Append epoch accuracy and loss
    val_accuracies.append(val_total_correct / len(val_dataset))
    val_losses.append(sum(val_total_loss) / len(val_total_loss))
    

# Print accuracies and losses per epoch
for i in range(epochs):
    print("Epoch", i + 1)
    print("Train acc and loss\t", train_accuracies[i], "\t", train_losses[i])
    print("Val acc and loss\t", val_accuracies[i], "\t", val_losses[i])

Training: 100%|██████████| 20/20 [00:01<00:00, 10.44it/s]

Epoch 1
Train acc and loss	 0.2075 	 2.2739684772491455
Val acc and loss	 0.215 	 2.1856272561209544
Epoch 2
Train acc and loss	 0.39 	 1.6707205581665039
Val acc and loss	 0.58 	 1.103754264967782
Epoch 3
Train acc and loss	 0.62875 	 0.9203212308883667
Val acc and loss	 0.61 	 0.9902970365115574
Epoch 4
Train acc and loss	 0.71625 	 0.7936684727668762
Val acc and loss	 0.68 	 0.8487601450511387
Epoch 5
Train acc and loss	 0.42875 	 1.6682796931266786
Val acc and loss	 0.25 	 1.811011825289045
Epoch 6
Train acc and loss	 0.43375 	 1.528413140773773
Val acc and loss	 0.545 	 1.3772424118859428
Epoch 7
Train acc and loss	 0.6475 	 1.2193508434295655
Val acc and loss	 0.78 	 0.7262568516390664
Epoch 8
Train acc and loss	 0.81125 	 0.6100793164968491
Val acc and loss	 0.865 	 0.4110597925526755
Epoch 9
Train acc and loss	 0.84875 	 0.43641544044017794
Val acc and loss	 0.86 	 0.30194943451455664
Epoch 10
Train acc and loss	 0.91375 	 0.326254115998745
Val acc and loss	 0.94 	 0.2179505649

In [61]:
with open('trainvalAccs/' + sfile + '.txt', 'w') as f:
    for i in range(epochs):
        f.write("Epoch " + str(i + 1) + "\n")
        f.write("Train acc and loss\t" + str(train_accuracies[i]) + "\t" + str(train_losses[i]) + "\n")
        f.write("Val acc and loss\t" + str(val_accuracies[i]) + "\t" + str(val_losses[i]) + "\n")

In [28]:
# Save the model
torch.save(model.state_dict(), 'testsave/' + sfile + '.pth')

In [7]:
# Layers of the model
model_layers = [28, "M", 55, "M", 111,"M"]
# Create model
model = small2DNet(model_layers, model_layers[-2], linear_neurons)
# Load model
model.load_state_dict(torch.load('testsave/'+ sfile + '.pth'))
# Put model on gpu
model.cuda()
hi = 5

In [20]:
# Testing
    
wrong_dict = {}
right_dict = {}

for i in range(10):
    wrong_dict[i] = {}
    for j in range(10):
        wrong_dict[i][j] = 0
    right_dict[i] = 0
    
for it in range(1):
    # Total and amount correct
    test_correct = 0
    test_total = 0

    # File to write predicted labels
    # with open("predicted_labels.txt", "w") as f:
    #     f.write("\n")


    # Put the model in evaluation mode
    model.eval()

    # Without gradient calculation
    with torch.no_grad():
        for (images, labels) in tqdm(test_dataloader):

            # Put images
            images = images.cuda()

            # Predicted labels
            preds = model(images)

            # Top predictions per image
            _, top_preds = torch.max(preds, 1)

            # Predictions and images back on cpu
            top_preds = top_preds.cpu()
            images = images.cpu()
            
            # Check the predicted
            for i in range(len(labels)):
                if top_preds[i].item() == labels[i].item():
                    right_dict[top_preds[i].item()] += 1
                else:
                    wrong_dict[labels[i].item()][top_preds[i].item()] += 1

            # Amount of correct predictions
            predictions = [top_preds[i].item() == labels[i].item() for i in range(len(labels))]
            correct = np.sum(predictions)

    #         # Show batch images
    #         fig, axs = plt.subplots(4,8, figsize=(28, 28), facecolor='w', edgecolor='k')
    #         fig.subplots_adjust(hspace = .5, wspace=.001)
    #         axs = axs.ravel()
    #         for i in range(len(images)):
    #             axs[i].imshow(images[i].permute(1, 2, 0))

    #         break

            # Update total correct and total images
            test_correct += correct
            test_total += len(images)


    print("Correct", test_correct, "/", test_total, "Accuracy:", test_correct / test_total)

100%|██████████| 313/313 [00:00<00:00, 486.96it/s]

Correct 9206 / 10000 Accuracy: 0.9206


In [ ]:
for x in right_dict:
    print(x, ":", right_dict[x])

In [ ]:
hm = []
for i, x in enumerate(wrong_dict):
    hm.append([])
    for y in wrong_dict[x]:
        hm[i].append(wrong_dict[x][y])
    # print(x, ":", wrong_dict[x])
print(hm)

In [ ]:
sns.heatmap(hm, cmap="Blues")

In [ ]:
arr1 = [-0.5428, -0.6003, -0.5327, -0.5052, -0.5986, -0.6092, -0.5588, -0.5108, -0.8328, -0.5751, -0.6494, -0.4935, -0.6055, -0.5924, -0.3553, -0.6655]
arr2 = [-0.5557, -0.5512, -0.5554, -0.9261, -0.5436, 0.9913, -0.5477, -0.5554, -0.5321, -0.5379, -0.5537, -0.5324, -0.5434, -0.5588, -0.5326, -0.5602]

print(np.round(sum(arr1) - sum(arr2), 4))


